# Amazon Reviews Dataset – Preprocessing, Transformation, and Visualization

This notebook performs the following tasks using **only** Python, `pandas`, `matplotlib`, and `seaborn`:

1. Data Preprocessing
   - Check for missing values in all columns.
   - If there are no missing values, introduce 5% missingness in rating and review text.
   - Handle missing values via imputation.
   - Clean review text (lowercase, remove punctuation, remove stopwords, tokenization).
   - Encode categorical variables such as product and user IDs.

2. Data Transformation
   - Convert review timestamps into datetime objects.
   - Derive new features like review year/month and rating categories (positive/negative/neutral).

3. Data Visualization
   - Visualize the distribution of review ratings.
   - Show the trend of number of reviews over time.
   - Plot missing value distributions before and after imputation.


## 0. Setup – Imports & File Path

Update `csv_path` to point to your CSV file (e.g. `"/content/train.csv"` in Google Colab).

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

sns.set(style="whitegrid")

# Path to your CSV file
csv_path = "/content/train.csv"  # TODO: change this if your file is elsewhere


## 1. Inspect Columns and Configure Logical Roles

We first load a small sample to see column names and preview the data. Then we map columns to logical roles like rating, text, product ID, user ID, and timestamp.


In [ ]:
# Load a small sample to inspect columns
sample = pd.read_csv(csv_path, nrows=5)
print("Columns in the dataset:")
print(sample.columns)
sample.head()


Now set the logical column names below. **Edit these strings** to match your actual dataset column names.

In [ ]:
# TODO: Edit these to match your dataset
RATING_COL  = "rating"      # e.g. "rating" or "Polarity"
TEXT_COL    = "Text"        # e.g. "Text" or "review_text"
PRODUCT_COL = "product_id"  # set to None if not available
USER_COL    = "user_id"     # set to None if not available
TIME_COL    = "timestamp"   # set to None if no time column


Load the full dataset. If you run into memory errors, you can later switch to chunked processing, but for now we assume it fits into memory.

In [ ]:
df = pd.read_csv(
    csv_path,
    header=None,
    engine='python',
    on_bad_lines='skip'   # In built Function for Skipping broken lines such as null or ""
)

print("Shape of full dataset:", df.shape)
df.head()


## 2. Check Missing Values in Original Data


In [ ]:
orig_missing = df.isna().sum()
print("Missing values in ORIGINAL dataset:")
print(orig_missing)
print("\nTotal missing:", int(orig_missing.sum()))


## 3. Introduce 5% Missingness (If None Exist)

If the dataset has no missing values at all, we randomly introduce 5% missingness into the rating and review text columns to simulate real-world data quality issues.

In [ ]:
np.random.seed(42)  # for reproducibility

# Work on a copy of the original data
df_work = df.copy()

if orig_missing.sum() == 0:
    n = len(df_work)

    # 5% masks for rating and text
    mask_rating = np.random.rand(n) < 0.05
    mask_text   = np.random.rand(n) < 0.05

    if RATING_COL in df_work.columns:
        df_work.loc[mask_rating, RATING_COL] = np.nan

    if TEXT_COL in df_work.columns:
        df_work.loc[mask_text, TEXT_COL] = np.nan

    print("Introduced 5% missingness into rating and text.")
else:
    print("Dataset already has missing values; not injecting extra.")

missing_before = df_work.isna().sum()
print("Missing values BEFORE imputation (after possible injection):")
print(missing_before)


## 4. Handle Missing Values (Imputation)

- Review text: fill with `'missing_review'`.
- Rating: fill with dataset median.
- Other numeric columns: fill with median.
- Other object columns: fill with `'missing_<col>'`.


In [ ]:
# Handle review text
if TEXT_COL in df_work.columns:
    df_work[TEXT_COL] = df_work[TEXT_COL].fillna("missing_review")

# Handle rating via median imputation
if RATING_COL in df_work.columns:
    median_rating = df_work[RATING_COL].median()
    df_work[RATING_COL] = df_work[RATING_COL].fillna(median_rating)

# Other numeric columns
num_cols = df_work.select_dtypes(include=["number"]).columns.tolist()
if RATING_COL in num_cols:
    num_cols.remove(RATING_COL)

for col in num_cols:
    col_median = df_work[col].median()
    df_work[col] = df_work[col].fillna(col_median)

# Other object columns
obj_cols = df_work.select_dtypes(include=["object"]).columns.tolist()
if TEXT_COL in obj_cols:
    obj_cols.remove(TEXT_COL)

for col in obj_cols:
    df_work[col] = df_work[col].fillna(f"missing_{col}")

missing_after = df_work.isna().sum()
print("Missing values AFTER imputation:")
print(missing_after)


## 5. Clean Review Text (Stopwords, Punctuation, Tokenization)

We convert text to lowercase, remove non-letter characters, split into tokens, remove basic English stopwords, and then join the tokens back into a cleaned string.

In [ ]:
BASIC_STOPWORDS = {
    "the", "is", "in", "and", "a", "an", "of", "to", "it", "this",
    "that", "i", "you", "for", "on", "with", "was", "are", "as",
    "but", "be", "have", "has", "at", "or", "so", "if"
}

def clean_text(text):
    text = str(text).lower()
    # keep only letters and spaces
    text = re.sub(r"[^a-z\s]", " ", text)
    tokens = text.split()
    tokens = [t for t in tokens if t not in BASIC_STOPWORDS]
    return " ".join(tokens)

if TEXT_COL in df_work.columns:
    df_work["clean_text"] = df_work[TEXT_COL].apply(clean_text)
    df_work[[TEXT_COL, "clean_text"]].head()
else:
    print("TEXT_COL not found; skipping text cleaning.")


## 6. Encode Categorical Variables (Product ID, User ID)

We convert product and user IDs to integer codes using pandas' `category` dtype.

In [ ]:
if PRODUCT_COL is not None and PRODUCT_COL in df_work.columns:
    df_work["product_id_enc"] = df_work[PRODUCT_COL].astype("category").cat.codes

if USER_COL is not None and USER_COL in df_work.columns:
    df_work["user_id_enc"] = df_work[USER_COL].astype("category").cat.codes

df_work.head()


## 7. Convert Timestamps & Derive New Features

We convert the timestamp column to a datetime type (if present), then derive review year, month, and a rating category (`positive`, `neutral`, `negative`).

In [ ]:
if TIME_COL is not None and TIME_COL in df_work.columns:
    df_work["review_datetime"] = pd.to_datetime(df_work[TIME_COL], errors="coerce")
    df_work["review_year"] = df_work["review_datetime"].dt.year
    df_work["review_month"] = df_work["review_datetime"].dt.month
else:
    print("No valid TIME_COL configured; skipping datetime features.")

def rating_category(x):
    try:
        x = float(x)
    except:
        return "unknown"
    if x >= 4:
        return "positive"
    elif x <= 2:
        return "negative"
    else:
        return "neutral"

if RATING_COL in df_work.columns:
    df_work["rating_category"] = df_work[RATING_COL].apply(rating_category)
    df_work[[RATING_COL, "rating_category"]].head()
else:
    print("No RATING_COL; skipping rating categories.")


## 8. Visualization – Distribution of Review Ratings


In [ ]:
if RATING_COL in df_work.columns:
    plt.figure(figsize=(6, 4))
    sns.countplot(x=RATING_COL, data=df_work)
    plt.title("Distribution of Review Ratings")
    plt.xlabel("Rating")
    plt.ylabel("Count")
    plt.show()
else:
    print("No RATING_COL to plot distribution.")


## 9. Visualization – Trend of Number of Reviews Over Time

We plot the monthly count of reviews over time if datetime information is available.

In [ ]:
if "review_datetime" in df_work.columns:
    df_work["year_month"] = df_work["review_datetime"].dt.to_period("M")
    trend = df_work.groupby("year_month").size().reset_index(name="review_count")

    plt.figure(figsize=(10, 4))
    plt.plot(trend["year_month"].astype(str), trend["review_count"], marker="o")
    plt.xticks(rotation=60)
    plt.title("Number of Reviews Over Time (Monthly)")
    plt.xlabel("Year-Month")
    plt.ylabel("Number of Reviews")
    plt.tight_layout()
    plt.show()
else:
    print("No 'review_datetime' column; cannot plot trend over time.")


## 10. Visualization – Missing Values Before vs After Imputation

We compare the missing value counts per column before and after imputation.

In [ ]:
missing_df = pd.DataFrame({
    "column": missing_before.index,
    "before": missing_before.values,
    "after": missing_after.values
})

missing_long = missing_df.melt(id_vars="column", value_vars=["before", "after"],
                               var_name="stage", value_name="missing_count")

plt.figure(figsize=(10, 4))
sns.barplot(data=missing_long, x="column", y="missing_count", hue="stage")
plt.title("Missing Values Before vs After Imputation")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()
